# Tiled Python Client Demonstration

Demonstrate a Python client accessing a tiled data server (running on `localhost`).  The server provides two databroker catalogs (`bdp2022` and `20idb_usaxs`).

Show two types of Python client:

- The `requests` package from the Python Standard Library.
- The `tiled` package from the Bluesky Framework.

For each type of client, show some specific queries and responses.

* [x] Find all runs in a catalog between these two ISO8601 dates.
* [x] Find run(s) which match given metadata.
* [x] Get overall metadata from given run.
* [x] What are the data streams in this run?
* [x] What is the metadata for this stream?
* [x] Get the data from the data stream named primary (the canonical main data).

## Client using `requests` package

Using the `requests` package, search the tiled server's  API using the `http://` interface by assembling a URI.  The tiled server will respond and we'll return the response as JSON.  We'll let Python handle report any Exceptions that might occur.

In [1]:
import requests

def requests_tiled(server, catalog, api="/api/v1/node/search", suffix="", port=8000):
    return requests.get(f"http://{server}:{port}{api}/{catalog}{suffix}").json()

As a convenience, make a function that converts a string representation of the date and time in ISO-8601 format into the Linux EPOCH floating-point representation needed for tiled's API.

In [2]:
import datetime

def iso_to_ts(isotime):
    return datetime.datetime.fromisoformat(isotime).timestamp()

We'll search the BDP project's databroker catalog, known to the tiled server (running on workstation `localhost`) by the text name `bdp2022`.

In [3]:
server = "localhost"
port = 8000
catalog = "bdp2022"

### Find runs within range of dates

Define the ends of the time span for the search query:

In [4]:
# Find all runs in a catalog between these two ISO8601 dates.
start_time = "2022-05-01"
end_time = "2022-11-01"
tz = "US/Central"

Using the `requests` package, ask the tiled server for all runs in the catalog that match the time range.

Here, we build up the URI suffix in parts to expose how the search query is constructed.  The response is a Python dictionary.  We won't print the entire dictionary here since it likely contains a lot of information, perhaps too much to show in full.

In [21]:
r = requests.get(
    (
        f"http://{server}:{port}"
        "/api/v1/node/search"
        f"/{catalog}"
        "?page[limit]=0"  # 0: all matching
        f"&filter[time_range][condition][since]={iso_to_ts(start_time)}"
        f"&filter[time_range][condition][until]={iso_to_ts(end_time)}"
        f"&filter[time_range][condition][timezone]={tz}"
        "&sort=time"
    )
).json()

Summarize the results (in object `r`):

In [6]:
print(f'Search of {catalog=} has {len(r["data"])} runs.')
xref = dict(First=0, Last=-1)
for k, v in dict(First=0, Last=-1).items():
    md = r["data"][v]["attributes"]["metadata"]
    # md keys: start  stop  summary
    # summary key is composed by tiled server
    plan_name = md["summary"]["plan_name"]
    scan_id = md["summary"]["scan_id"]
    started = md["summary"]["datetime"]
    print(f"{k:5s} run: {started=} {scan_id=} {plan_name=}")

Search of catalog='bdp2022' has 397 runs.
First run: started='2022-05-03T08:37:21.510276' scan_id=1596 plan_name='take_image'
Last  run: started='2022-09-08T13:54:25.178280' scan_id=1960 plan_name='push_images'


### Find runs matching a given plan name

Find run(s) which match some given metadata.  In this search, let's find all the runs that match a given `plan_name`.  Let's use the `take_image` plan from the previous results.

In [22]:
plan_name = "take_image"
print(f"Search for {plan_name=}")

r = requests.get(
    (
        f"http://{server}:{port}"
        "/api/v1/node/search"
        f"/{catalog}"
        "?page[limit]=0"  # 0: all matching
        "&filter[eq][condition][key]=plan_name"
        f'&filter[eq][condition][value]="{plan_name}"'
        "&sort=time"
    )
).json()

Search for plan_name='take_image'


In [23]:
print(f'Search of {catalog=} has {len(r["data"])} runs.')
xref = dict(First=0, Last=-1)
for k, v in dict(First=0, Last=-1).items():
    md = r["data"][v]["attributes"]["metadata"]
    # md keys: start  stop  summary
    # summary key is composed by tiled server
    plan_name = md["summary"]["plan_name"]
    scan_id = md["summary"]["scan_id"]
    started = md["summary"]["datetime"]
    print(f"{k:5s} run: {started=} {scan_id=} {plan_name=}")

Search of catalog='bdp2022' has 1029 runs.
First run: started='2022-03-22T16:48:41.130881' scan_id=1 plan_name='take_image'
Last  run: started='2022-08-30T15:06:37.662096' scan_id=1959 plan_name='take_image'


### Show a run's metadata

Let's show the various metadata available from a Bluesky *run*.  We'll use the last run from the previous search.

In [9]:
run = r["data"][-1]  # most recent run from previous results

The `run` object is a dictionary.  The interesting keys are:

key | content
:--- | :---
`id` | `uid` universal identifier of this `run` (used by the database)
`attributes` | contents of this `run`

The `attributes` contents are a dictionary with these interesting keys (there are other keys, as well):

key | content
:--- | :---
`metadata` | metadata dictionary of this `run`

The `metadata` dictionary has these keys:

key | content
:--- | :---
`start` | Metadata created as the run started (includes user-supplied, scan-specific, facility-specific, and bluesky metadata).  The `start` dictionary keys will vary between runs and catalogs.  Only a few are expected, including: `uid`, `time`, & `versions`.
`stop` | Metadata about how the run ended (exit status and reason if problem, stream names, end time stamp)
`summary` | tiled server provides this additional high-level summary with ISO8601 start date and run duration

Note: the run's data streams are obtained by a different query, using the run's `uid`.  Keep track of the `uid` for that reason.

To show the structure of this dictionary, we just access Python to show the object's value.

In [10]:
run["attributes"]["metadata"]

{'start': {'uid': 'a4edf4b3-8a12-4724-b817-fd45958488da',
  'time': 1661889997.662096,
  'versions': {'apstools': '1.6.2',
   'bluesky': '1.8.3',
   'bluesky_queueserver': '0.0.15',
   'databroker': '1.2.5',
   'epics': '3.5.0',
   'h5py': '3.7.0',
   'matplotlib': '3.5.2',
   'numpy': '1.20.3',
   'ophyd': '1.6.4',
   'pyRestTable': '2020.0.3',
   'spec2nexus': '2021.2.1'},
  'databroker_catalog': 'bdp2022',
  'login_id': 'bdp@terrier.xray.aps.anl.gov',
  'beamline_id': 'BDP',
  'instrument_name': 'APS-U Beamline Data Pipelines project in 2022',
  'proposal_id': 'bdp2022',
  'milestone': 'BDP M4 demo',
  'pid': 22441,
  'scan_id': 1959,
  'sample': 'simulated_sample_102_66',
  'game_key': 'fine image',
  'plan_type': 'generator',
  'plan_name': 'take_image',
  'detectors': ['adsimdet'],
  'num_points': 1,
  'num_intervals': 0,
  'plan_args': {'detectors': ["MySimDetector(prefix='bdpSimExample:', name='adsimdet', read_attrs=['hdf1', 'pva'], configuration_attrs=['cam', 'cam.acquire_peri

### Search for runs containing given text.

It is possible to search a catalog for runs containing given text.  Here is one example searching for `M9` (upper or lower case):

In [11]:
search_text = "M9"
case_sensitive = True
r = requests.get(
    (
        f"http://{server}:{port}"
        "/api/v1/node/search"
        f"/{catalog}"
        "?page[limit]=0"  # 0: all matching
        f"&filter[fulltext][condition][text]={search_text}"
        f"&filter[fulltext][condition][case_sensitive]={str(case_sensitive).lower()}"
        "&sort=time"
    )
).json()

In [12]:
print(f'Search of {catalog=} has {len(r["data"])} runs which contain "{search_text}".')
xref = dict(First=0, Last=-1)
for k, v in dict(First=0, Last=-1).items():
    md = r["data"][v]["attributes"]["metadata"]
    # md keys: start  stop  summary
    # summary key is composed by tiled server
    plan_name = md["summary"]["plan_name"]
    scan_id = md["summary"]["scan_id"]
    started = md["summary"]["datetime"]
    print(f"{k:5s} run: {started=} {scan_id=} {plan_name=}")

Search of catalog='bdp2022' has 75 runs which contain "M9".
First run: started='2022-11-11T01:34:27.938719' scan_id=1961 plan_name='m9_push_images'
Last  run: started='2022-11-23T11:17:32.495794' scan_id=2035 plan_name='m9_push_images'


### What data streams are available with this run?

Use the last `take_image` run.  The stream names are in the `stop` metadata, where the number of data events is shown for each stream.

In [20]:
r = requests.get(
    (
        f"http://{server}:{port}"
        "/api/v1/node/search"
        f"/{catalog}"
        "?page[limit]=0"  # 0: all matching
        "&filter[eq][condition][key]=plan_name"
        '&filter[eq][condition][value]="take_image"'
        "&sort=time"
    )
).json()
stop_md = r["data"][-1]["attributes"]["metadata"]["stop"]
streams = list(stop_md["num_events"].keys())
uid = stop_md["run_start"]
print(f'Run {uid=} has {streams=}')

Run uid='a4edf4b3-8a12-4724-b817-fd45958488da' has streams=['primary']


### What is the metadata for the `primary` stream of this run?

In [14]:
stream_name = "primary"
r = requests_tiled(
    server, catalog,
    api="/api/v1/node/metadata",
    suffix=(
        f"/{uid}"
        f"/{stream_name}"
    )
)
print(f"{catalog=} {uid=} metadata for {stream_name=}:")
r['data']['attributes']['metadata']

catalog='bdp2022' uid='a4edf4b3-8a12-4724-b817-fd45958488da' metadata for stream_name='primary':


{'descriptors': [{'run_start': 'a4edf4b3-8a12-4724-b817-fd45958488da',
   'time': 1661889999.655602,
   'data_keys': {'adsimdet_image': {'shape': [1, 1024, 1024],
     'source': 'PV:bdpSimExample:',
     'dtype': 'array',
     'external': 'FILESTORE:',
     'object_name': 'adsimdet'}},
   'uid': 'f78f5d29-36dc-49e2-9e9a-8eee6120873b',
   'configuration': {'adsimdet': {'data': {'adsimdet_cam_acquire_period': 0.251,
      'adsimdet_cam_acquire_time': 0.25,
      'adsimdet_cam_image_mode': 0,
      'adsimdet_cam_manufacturer': 'Simulated detector',
      'adsimdet_cam_model': 'Basic simulator',
      'adsimdet_cam_num_exposures': 1,
      'adsimdet_cam_num_images': 1,
      'adsimdet_cam_trigger_mode': 0},
     'timestamps': {'adsimdet_cam_acquire_period': 1661889997.531498,
      'adsimdet_cam_acquire_time': 1661889997.520722,
      'adsimdet_cam_image_mode': 1661889997.5001307,
      'adsimdet_cam_manufacturer': 1661889153.029876,
      'adsimdet_cam_model': 1661889153.029901,
      'ad

### Get the data from the data stream named primary (the canonical main data).

To get the data, we need to change the type of search using `/api/v1/node/full` (so far, we have been using the default search for metadata: `/api/v1/node/search`) and specify the format of the result.  One format is `json`.  Let's pick some data from the M6 demo (with `M6-gallery` matching text) that has a very long list (1-D) of floating point numbers.

In [15]:
search_text = "M6-gallery"
case_sensitive = True
r = requests_tiled(
    server, catalog, suffix=(
        "?page[limit]=0"  # 0: all matching
        f"&filter[fulltext][condition][text]={search_text}"
        f"&filter[fulltext][condition][case_sensitive]={str(case_sensitive).lower()}"
        "&sort=time"
    )
)

In [16]:
uid = r["data"][-1]["attributes"]["metadata"]["start"]["uid"]
data_format = "json"
stream_name = "primary"
data_name = "adpvadet_pva1_execution_time"

arr= requests_tiled(
    server, catalog,
    api="/api/v1/array/full",
    suffix=(
        f"/{uid}"
        f"/{stream_name}"
        "/data"
        f"/{data_name}"
        f"?format={data_format}"
    )
)

In [17]:
print(f"{len(arr)=} {min(arr)=} {max(arr)=}")

len(arr)=10261 min(arr)=0.213429 max(arr)=2.024461


## Client using `tiled` package

In [18]:
from tiled.client import from_uri
from tiled.client.cache import Cache
import tiled.queries
from tiled.utils import tree